### Chapter4.구조적 API 개요

##### 구조적 API
데이터 흐름을 정의하는 기본 추상화 개념.
- Dataset(타입형/4.3.1부분 언급)
- DataFrame(비타입형/4.3.1부분 언급+다소 부정확할 수도 있음.)
- SQL 테이블과 뷰

*배치(batch) & 스트리밍(streaming) 처리에서 구조적 API 사용 가능 -> 관련 내용은 5부에서 자세히 다룰 예정

<해당 챕터 중점 내용>
- 타입형(typed) / 비타입형(untyped) API의 개념과 차이점
- 핵심 용어
- 스파크가 구조적 API의 데이터 흐름을 해석하고 클러스터에서 실행하는 방식

#### 4.1 DataFrame과 Dataset
- 분산테이블 형태의 컬렉션
- '스키마' 분산 컬렉션에 저장할 데이터 타입을 정의하는 방법

#### 4.2 스키마
- DataFrame의 컬럼명과 데이터 타입을 정의
- 데이터 소스에서 얻거나(schema-on-read) 직접 정의
- 어떤 데이터 타입이 어느 위치에 있는지 정의하는 방법 필요

#### 4.3 스파크의 구조적 데이터 타입 개요
- 프로그래밍 언어
- 카탈리스트 엔진 사용(실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 가짐)
- 파이썬이나 R을 이용해 스파크의 구조적 API를 사용하더라도 대부분의 연산은 스파크의 데이터 타입을 사용 
=> 스파크가 지원하는 언어를 이용해 작성된 표현식을 카탈리스트 엔진에서 스파크의 테이터 타입으로 변환하여 명령을 처리하기 때문

In [1]:
df = spark.range(500).toDF("number")

In [2]:
df.select(df["number"] + 10) #스파크의 덧셈 연산을 수행

DataFrame[(number + 10): bigint]

In [4]:
#df.select(df["number"] + 10).take(3)

[Row((number + 10)=10), Row((number + 10)=11), Row((number + 10)=12)]

In [12]:
df.take(3)

[Row(number=0), Row(number=1), Row(number=2)]

##### 4.3.1 DataFrame과 Dataset 비교
- DataFrame : 비타입형
    - 데이터 타입의 일치 여부를 '런타임'이 되어서야 확인
    - 스파크의 최적화된 내부 포맷을 사용할 수 있다. 
    - Row 타입으로 구성된 Dataset
    - Row 타입
        - '연산에 최적화된 인메모리 포맷'의 내부적인 표현 방식
        - 자체 데이터 포맷을 사용하기 때문에 효율적인 연산이 가능.

- Dataset : 타입형
    - 데이터 타입의 일치 여부를 '컴파일 타임'에 확인
    - JVM 기반의 언어인 스칼라와 자바에서만 지원
    - Dataset의 데이터 타입 정의를 위해서는 스칼라의 케이스 클래스나 자바 빈을 사용해야 한다.

##### 4.3.2 컬럼
- 단순 데이터 타입 / 복합 데이터 타입 / null 값
- 5장에서 자세히

##### 4.3.3 로우
- 데이터 레코드
- 데이터 소스에서 얻거나 직접 생성 가능(ex.range 메소드 등?)

In [14]:
spark.range(2).collect() #range 메서드로 DataFrame 생성

[Row(id=0), Row(id=1)]

##### 4.3.3 스파크 데이터 타입
스파크 지원 언어별 매핑 정보 참고: https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [10]:
from pyspark.sql.types import *

In [11]:
b = ByteType()

#### 4.4 구조적 API의 실행 과정
1. DataFrame/Dataset/SQL을 이용해 코드를 작성
2. 정상적인 코드라면 스파크가 논리적 실행 계획으로 변환
3. 스파크는 논리적 실행 계획을 물리적 실행 계획으로 변환하며, 그 과정에서 추가적 최적화 가능 여부 확인
4. 클러스터에서 물리적 실행 계획(RDD 처리)를 실행
<img src="../../assets/presentations/week04/catalyst_optimizer.png" width="500px" height="250px" title="catalyst_optimizer"/>

##### 4.4.1 논리적 실행 계획
<img src="../../assets/presentations/week04/the_structured_API_logical_planning_process.png" width="600px" height="300px" title="the_structured_API_logical_planning_process"/>

- 추상적 트랜스포메이션만 표현
- 드라이버나 익스큐터의 정보를 고려하지 않음
- 검증 전 논리적 실행 계획으로 변환

##### 4.4.2 물리적 실행 계획
<img src="../../assets/presentations/week04/the_physical_planning_process.png" width="700px" height="350px" title="the_physical_planning_process"/>

- 클러스터 환경에서 실행하는 방법을 정의
- 사용하려는 테이블의 크기, 파티션 수 등 물리적 속성을 고려해 지정된 조인 연산 수행에 필요한 비용을 계산하고 비교
##### 4.4.3 실행 